In [ ]:
import numpy as np
import h5py
import cv2
import os, sys
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.ndimage import median_filter
from skimage.metrics import *
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
%load_ext autoreload
%autoreload 2

In [ ]:
peak_signal_noise_ratio?

In [ ]:
!ls ../data/val_Samsung_SNU_patches_SET7_divided_by_fnum.hdf5


In [ ]:
samsung_images = {}
with h5py.File("../data/val_Samsung_SNU_patches_SET7_divided_by_fnum.hdf5", 'r') as f:
    f1_images = np.array(f['F1'])
    f2_images = np.array(f['F2'])
    f4_images = np.array(f['F4'])
    f8_images = np.array(f['F8'])
    f16_images = np.array(f['F16'])
    f32_images = np.array(f['F32'])
    f64_images = np.array(f['F64'])
    for f_num in ['F1', 'F2', 'F4', 'F8', 'F16', 'F32', 'F64']:
        samsung_images[f_num] = np.array(f[f_num])

In [ ]:
for key,item in samsung_images.items():
    print(key,item.shape)

In [ ]:
def apply_median_filter(img : np.array ,kernel_size  : tuple = (11,11), repeat: int =3 ,plot : bool =False):
    out = deepcopy(img)
    if plot is True:
        plt.title(f'before iter')
        plt.imshow(out[:200,:200])
        plt.pause(0.01)
        plt.figure(figsize=(20,4))
        plt.plot(out[50][:200])
        plt.pause(0.01)
    for i in range(repeat):
        out = median_filter(out,kernel_size)
        if plot is True:
            plt.title(f'{i+1} iter')
            plt.imshow(out[:200,:200])
            plt.pause(0.01)
            plt.figure(figsize=(20,4))
            plt.plot(out[50][:200])
            plt.pause(0.01)
    return out

In [ ]:
for f_num in ['F1', 'F2', 'F4', 'F8', 'F16', 'F32']:
    avg_psnr = []
    avg_ssim = []
    for noisy, clean in zip(samsung_images[f_num], samsung_images['F64']):
        denoised_img = apply_median_filter(noisy)
        avg_psnr.append(psnr(clean,denoised_img))
        avg_ssim.append(ssim(clean,denoised_img))
    print(f"==== {f_num} denoised result ====")
    print(f"PSNR : {np.mean(avg_psnr):.6f} SSIM : {np.mean(avg_ssim):.6f}")
    print(f"{np.mean(avg_psnr):.6f}\t{np.mean(avg_ssim):.6f}")

In [1]:
import pandas as pd
pd.DataFrame.from_dict?

Signature:
pd.DataFrame.from_dict(
    data,
    orient: 'str' = 'columns',
    dtype: 'Dtype | None' = None,
    columns=None,
) -> 'DataFrame'
Docstring:
Construct DataFrame from dict of array-like or dicts.

Creates DataFrame object from dictionary by columns or by index
allowing dtype specification.

Parameters
----------
data : dict
    Of the form {field : array-like} or {field : dict}.
orient : {'columns', 'index', 'tight'}, default 'columns'
    The "orientation" of the data. If the keys of the passed dict
    should be the columns of the resulting DataFrame, pass 'columns'
    (default). Otherwise if the keys should be rows, pass 'index'.
    If 'tight', assume a dict with keys ['index', 'columns', 'data',
    'index_names', 'column_names'].

    .. versionadded:: 1.4.0
       'tight' as an allowed value for the ``orient`` argument

dtype : dtype, default None
    Data type to force, otherwise infer.
columns : list, default None
    Column labels to use when ``orient='index'``

In [ ]:
images_denoised_with_median_filter = {}